## [Demo] Working with PSANA inside Jupyter lab notebook

This notebook starts a Dask scheduler that serves as a bridge to share data between PSANA and Jupyter Lab. Using the `slurm_magic` package, users can submit a PSANA job and return results to be manipulated within the notebook. 

**Note**: `%srun` is a blocking magic command. 

In [1]:
print("remove # to clean")
!rm -rf *.json *.log .tmp

remove # to clean


Load the `slurm_magic` package

In [2]:
%load_ext slurm_magic
# %squeue -u USERNAME

We proceed to start a scheduler in the login node to be our bridge to share data. 

In [3]:
from dask.distributed import Client, LocalCluster, Queue

In [4]:
scheduler_file = 'scheduler.json'
cluster = LocalCluster(n_workers=1, threads_per_worker=1, host="0.0.0.0")
client = Client(cluster)
client.write_scheduler_file(scheduler_file)

We can now verify our scheduler is up and running. We also created a `scheduler.json` file that can be used to connect as many notebooks as desired.

In [5]:
client

Client Scheduler: tcp://128.55.224.49:44499 Dashboard: http://128.55.224.49:8787/status,Cluster Workers: 1 Cores: 1 Memory: 540.15 GB


In [6]:
import os 

if os.path.isfile(scheduler_file):
    print("Scheduler file with name {} exists!".format(scheduler_file))

Scheduler file with name scheduler.json exists!


Now that client is up and running, there are potentially two ways to submit and gather data from a PSANA job:

1. Using a `Queue` in Dask. 
2. Using Pub/Sub scheme. 

## Queue example

In [7]:
!python /global/common/software/lcls/psana/lcls2/psana/psana/tests/setup_input_files.py


Start writing offsets.
Finished writing smd for 9 L1 events and 8 update events. Big data file has 19 events with size (B): 37120

Start writing offsets.
Finished writing smd for 9 L1 events and 8 update events. Big data file has 19 events with size (B): 32728


In [8]:
%srun -n 3 -A m3384  -C haswell  -q debug  -o test.log python test.py &

('',
 'Launched in background. Redirecting stdin to /dev/null\nsrun: job 26816514 queued and waiting for resources\nsrun: job 26816514 has been allocated resources\n')

In [9]:
# %squeue -u USERNAME

In [10]:
q = Queue("psana")

In [11]:
import numpy as np

results = []
for data in range(q.qsize()):
    results.append(np.array(q.get()))

In [12]:
results

[array([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],
 
        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],
 
        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],
 
        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]]]), array([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],
 
        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],
 
        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],
 
        [[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]]]), array([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],
 
        [[ 0,  1,  2,  3,  4, 